## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [2]:
# 讀取鳶尾花資料集
iris = datasets.load_iris()

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.25, random_state=34)

# 建立模型  參數 = (max_depth=1, min_samples_split=2)
clf = DecisionTreeClassifier(max_depth=1, min_samples_split=2, random_state=2)

In [3]:
#得到的結果，約為 0.55263 的 accuracy 
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

acc = metrics.accuracy_score(y_test, y_pred)
print('預設參數的 accuracy = %.5f'%acc)

預設參數的 accuracy = 0.55263


In [4]:
# 設定要訓練的超參數組合
max_depth = [1, 2, 3, 4]
min_samples_split  = [2, 3, 4, 5]
param_grid = dict(min_samples_split=min_samples_split, max_depth=max_depth)
print(param_grid)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 12 種參數組合，總共要 train 36 次模型

{'min_samples_split': [2, 3, 4, 5], 'max_depth': [1, 2, 3, 4]}
Fitting 3 folds for each of 16 candidates, totalling 48 fits


c:\users\raoblack\python_env\100ai\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 out of  48 | elapsed:    1.0s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:    1.0s finished
c:\users\raoblack\python_env\100ai\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [5]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -0.080357 using {'max_depth': 2, 'min_samples_split': 2}


In [6]:
grid_result.best_params_

{'max_depth': 2, 'min_samples_split': 2}

In [7]:
# 使用最佳參數重新建立模型
clf_bestparam = DecisionTreeClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           min_samples_split=grid_result.best_params_['min_samples_split'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

acc = metrics.accuracy_score(y_test, y_pred)
print('調整參數後的 accuracy = %.5f'%acc)

調整參數後的 accuracy = 0.92105
